### Init CosmWasm Client

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";
import { SigningCosmWasmClient } from "@cosmjs/cosmwasm-stargate"
import { coins, GasPrice } from '@cosmjs/stargate';


if(simulate) {
  senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
  client = new SimulateCosmWasmClient({
    chainId: "Oraichain",
    bech32Prefix: "orai",
    metering: true
  });
  client.app.bank.setBalance(senderAddress, coins('10000000', 'orai'));
} else {  
  await keplr.enable('Oraichain-testnet');
  const signer = await keplr.getOfflineSignerAuto('Oraichain-testnet');
  senderAddress = (await signer.getAccounts())[0].address;
  client = await SigningCosmWasmClient.connectWithSigner('https://testnet.rpc.orai.io', signer, {  
      gasPrice: GasPrice.fromString('0.002orai')  
  });
}

console.log(await client.getBalance(senderAddress, 'orai'));

### Deploy contract on SimulateCosmwasmClient

In [ ]:
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/cw20-base.wasm").then((res) => res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(
  senderAddress,
  codeId,
  {
    decimals: 6,
    symbol: "CVT",
    name: "cosmoverse token",
    mint: { minter: senderAddress },
    initial_balances: [{ address: senderAddress, amount: "1000000000" }]
  },
  "cw20 token",
  "auto"
);

cw20Contract = new Cw20BaseClient(client, senderAddress, contractAddress);

console.log(await cw20Contract.tokenInfo());

### Execute contract with gas metering

In [ ]:
const { gasUsed } = await cw20Contract.transfer({amount: '1000000', recipient: 'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'});
console.log(`Transfer token required gas: ${gasUsed}`);

console.log(await cw20Contract.balance({address:'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'}));